# Median filter

### Для начала на обычное изображение наложим шум типа "соль и перец", и затем уже с ним будем работать

In [1]:
import numpy as np
import random
import cv2

#метод для создания шума
def noise_sp(image,v):
    v2 = 1 - v 
    output = np.zeros(image.shape,np.uint8)
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            rand = random.random()
            if rand < v:
                output[i,j] = 0
            elif rand > v2:
                output[i,j] = 255
            else:
                output[i,j] = image[i][j]
    return output

image_or = cv2.imread('sample_data/Emilia.bmp',0)
image = noise_sp(image_or,0.05)
cv2.imwrite('sample_data/Emilia_noise.bmp', image)

True

In [2]:
#Изображение с шумом
image = cv2.imread('sample_data/Emilia_noise.bmp', cv2.IMREAD_GRAYSCALE)

### Реализуем медианный фильтр на CPU

In [3]:
#Метод для рассчёта медианного фильтра
def SAP(image):

    result = np.zeros(image.shape)
    pr_result=np.zeros(9)
    
    for a in range(1, image.shape[0]-1):
        for b in range(1, image.shape[1]-1):

            pr_result[0:3]=image[a-1:a+2,b]
            pr_result[3:6]=image[a-1:a+2,b+1]
            pr_result[6:9]=image[a-1:a+2,b-1]

            p=sorted(pr_result)

            result[a, b] = p[4]

    #Обрабатываем граничные пиксели      
    result[0,:]=result[1,:]
    result[image.shape[0]-1,:]=result[image.shape[0]-2,:]
    result[:,0]=result[:,1]
    result[:,image.shape[1]-1]=result[:,image.shape[1]-2]
    
    return result

In [4]:
#Обрабатываем изображение на CPU
import time
start1 = time.time()

result_image=SAP(image)

end1 = time.time() - start1

In [5]:
#Записываем полученное изображение
cv2.imwrite('sample_data/result1.bmp', result_image)

True

In [6]:
#Выводим время работы на cpu
print("Время обработки изображения на CPU -", end1)

Время обработки изображения на CPU - 1.495190143585205


### Реализуем медианный фильтр на GPU

In [7]:
#Устанавливаем pycuda
!pip install pycuda

In [8]:
from pycuda import driver, compiler
import pycuda.autoinit
from pycuda.compiler import SourceModule
BLOCK_SIZE = 16

In [9]:
#Пишем функцию ядра и вспомогательную, которая будет производить сортировку

calculate_bilateral_GPU = SourceModule("""
texture<unsigned int, 2> tex;

//сортировка массива 
__device__ inline void Sort_m(int massn[], int n) {
    for (int i = 0; i < n; i++) {
        bool flag = true;
        int r;
        for (int j = 0; j < n - (i + 1); j++) { 
            if (massn[j] > massn[j + 1]) {
                flag = false;
                r=massn[j];
                massn[j]=massn[j + 1];
                massn[j + 1]=r;
            }
        }
        if (flag) break;
      }
}

//Функция ядра
__global__ void kernel(unsigned int * __restrict__ image,const int M, const int N)
{
    //получаем номер нити
    const int x = threadIdx.x + blockDim.x * blockIdx.x;
    const int y = threadIdx.y + blockDim.y * blockIdx.y;
    
    //проверяем что не вышли за рамки изображения
    if ((x < M) && (y < N)) {
      int massn[9];
      int n=0;

      for (int a = x-1; a <= x+1; a++){
        for (int b = y-1; b <= y+1; b++){
          //сохраняем центральный пиксель и окружающие его в массив
          massn[n]=tex2D(tex, b, a);
          n++;
        }
      }
        //сортируем массив
        Sort_m(massn,9);
        //И в результирующий пиксель записываем значение с середины массива
        image[x*N + y] = massn[4];
    }
}""")

kernel = calculate_bilateral_GPU.get_function("kernel")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: UserWarning: The CUDA compiler succeeded, but said the following:
nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).



In [10]:
#Задаём сетку
M,N=image.shape
result_image2 = np.zeros((M,N), dtype=np.uint32)
grid = (int(np.ceil(M/BLOCK_SIZE)),int(np.ceil(N/BLOCK_SIZE)))

start2 = time.time()

#Копируем данные в текстуру
cu_tex = calculate_bilateral_GPU.get_texref("tex")
driver.matrix_to_texref(image.astype(np.uint32), cu_tex, order="C")

#Запускаем функцию ядра
kernel(driver.Out(result_image2), np.int32(M), np.int32(N), texrefs=[cu_tex], block=(BLOCK_SIZE,BLOCK_SIZE,1), grid=grid)

end2 = time.time() - start2

#Сохраняем новое изображение
cv2.imwrite('sample_data/result2.bmp', result_image2.astype(np.uint8))

True

In [11]:
print("Время обработки изображения на GPU -", end2)

Время обработки изображения на GPU - 0.0033969879150390625
